In [2]:
#Grab some relevant part of util.py in qtrkx-gnn-tracking

import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
#internal
import os, sys, glob, yaml, datetime, argparse
import csv
import tensorflow as tf

Graph = namedtuple('Graph', ['X', 'Ri', 'Ro', 'y'])

class GraphDataset():
    def __init__(self, input_dir, n_samples=None):
        input_dir = os.path.expandvars(input_dir)
        filenames = [os.path.join(input_dir, f) for f in os.listdir(input_dir)
                     if f.startswith('event') and f.endswith('.npz')]
        self.filenames = (
            filenames[:n_samples] if n_samples is not None else filenames)

    def __getitem__(self, index):
        return load_graph(self.filenames[index])

    def __len__(self):
        return len(self.filenames)
def get_dataset(input_dir,n_files):
    return GraphDataset(input_dir, n_files)
def load_graph(filename):
    """Reade a single graph NPZ"""
    with np.load(filename) as f:
        return sparse_to_graph(**dict(f.items()))
def sparse_to_graph(X, Ri_rows, Ri_cols, Ro_rows, Ro_cols, y, dtype=np.float32):
    n_nodes, n_edges = X.shape[0], Ri_rows.shape[0]
    Ri = np.zeros((n_nodes, n_edges), dtype=dtype)
    Ro = np.zeros((n_nodes, n_edges), dtype=dtype)
    Ri[Ri_rows, Ri_cols] = 1
    Ro[Ro_rows, Ro_cols] = 1
    return Graph(X, Ri, Ro, y)

In [3]:
current_directory = os.getcwd()

In [4]:
get_dataset(current_directory,1)

In [9]:
#Read one event
graph_ex=load_graph('event000001000_g000.npz')

In [19]:
#Load raw data
X=graph_ex.X
Ri=graph_ex.Ri
Ro=graph_ex.Ro
y=graph_ex.y

In [16]:
print("X shape: ", X.shape," Ri shape: ", Ri.shape,
      " Ro shape: ", Ro.shape," y shape: ", y.shape)

X shape:  (6437, 3)  Ri shape:  (6437, 10670)  Ro shape:  (6437, 10670)  y shape:  (10670,)


In [26]:
#Need to change type for later use.
y=y.astype(np.int32)
Ri = Ri.astype(np.int32)
Ro = Ro.astype(np.int32)

In [33]:
y.sum()

5162

In [ ]:
import numpy as np

def graph_coloring(Ri, Ro, y):
    # Get the number of nodes and edges
    NV, NE = Ri.shape

    # Initialize the coloring vector
    coloring = np.zeros(NV, dtype=int)

    # Create a dictionary to store the neighbors of each node
    neighbors = {i: [] for i in range(NV)}
    print("Start getting neighborhood information")
    for k in range(NE):
        if k%2000==0: 
            print("Performing ",k,"-th iteration...")
        if y[k] == 1:
            for j in range(NV):
                if Ro[j, k] == 1:
                    for i in range(NV):
                        if Ri[i, k] == 1 and i != j:
                            neighbors[j].append(i)
    print("Start greedy coloring.")
    # Greedy coloring
    for node in range(NV):
        if node%1000==0:
            print("Performing ",node,"-th iteration...")
        used_colors = set(coloring[n] for n in neighbors[node])
        # Find the smallest unused color
        color = 0
        while color in used_colors:
            color += 1
        coloring[node] = color
    return coloring

coloring = graph_coloring(Ri, Ro, y)
print(coloring)

Start getting neighborhood information
Performing  0 -th iteration...


In [36]:
coloring.sum()

5162

In [37]:
y

array([1, 0, 0, ..., 1, 1, 1], dtype=int32)

In [38]:
y.sum()

5162